In [47]:
import sys
from PyQt5 import QtCore, QtWidgets
from PyQt5.QtWidgets import QMainWindow, QCheckBox,  QApplication, QWidget, QPushButton, QAction, QLineEdit, QMessageBox, QInputDialog, QLabel
from PyQt5.QtGui import *  
from PyQt5.QtCore import QSize 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import pandas_datareader.data as web
import datetime as dt
import bs4 as bs
import pickle
import requests
import os
import time
import numpy as np
from scipy.interpolate import UnivariateSpline
%matplotlib inline

style.use('ggplot')

In [6]:
def create_csv(stock='TSLA',start=dt.datetime(2017,1,1)):
    
    '''This function takes values from yahoo finance and turns it into a pandas database, saving it as a csv in a folder'''

    now = dt.datetime.now()
    end=dt.datetime(now.year,now.month,now.day)
    
    dataframe = web.DataReader(stock,'yahoo',start,end)
    
    TSLA = dataframe.to_csv('stock_dfs/{}.csv'.format(stock))

In [7]:
def read_csv(stock='TSLA'):
    
    '''This function reads the data from the csv stored by create_csv and adds some rolling averages and plots it all'''
    
    fit = plt.figure(figsize = [20,10])
    df=pd.read_csv('stock_dfs/{}.csv'.format(stock),parse_dates=True,index_col=0)
    
    df['300ma']=df['Adj Close'].rolling(window=300,min_periods=0).mean()
    df['100ma']=df['Adj Close'].rolling(window=100,min_periods=0).mean()
    df['40ma']=df['Adj Close'].rolling(window=40,min_periods=0).mean()
    df['20ma']=df['Adj Close'].rolling(window=20,min_periods=0).mean()
    
    ax1=plt.subplot2grid((6,3),(0,0),rowspan=5,colspan=3,ylabel='Stock Price',title=stock)
    ax2=plt.subplot2grid((6,3),(5,0),rowspan=1,colspan=3,sharex=ax1)
    
    ax1.plot(df.index,df['Adj Close'])
    ax1.plot(df.index,df['300ma'],label="300 day")
    ax1.plot(df.index,df['100ma'],label="100 day")
    ax1.plot(df.index,df['40ma'],label="40 day")
    ax1.plot(df.index,df['20ma'],label="20 day")
    ax1.legend(loc='upper right')
    ax2.plot(df.index,df['Volume'],color='black')
    df.to_csv('stock_dfs/{}.csv'.format(stock))
    

In [8]:
def save_sp500_tickers():
    
    '''This function takes the S&P 500 list and creates csvs for each of them using beautiful soup'''
    
    resp=requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text,'lxml')
    table = soup.find('table',{'class':'wikitable sortable'})
    stock_names = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        stock_names.append(ticker)
            
    with open('sp500tickers.pickle','wb') as f:
        pickle.dump(stock_names,f)
        
    print(stock_names)
    
    return stock_names

In [9]:
def get_data_from_yahoo(reload_sp500=False):
    
    '''This reimports data for the S&P 500 companies if needed'''
    
    if reload_sp500==True:
        tickers = save_sp500_tickers()
    else:
        with open('sp500tickers.pickle','rb') as f:
            tickers = pickle.load(f)
            
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
    
    start=dt.datetime(2017,1,1)
    now = dt.datetime.now()
    end=dt.datetime(now.year,now.month,now.day)
    
    for ticker in tickers[:100]:
        try:
            if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
                df=web.DataReader(ticker,'yahoo',start,end)
                df.to_csv('stock_dfs/{}.csv'.format(ticker))
            #else:
                #print('Already have {}'.format(ticker))
        except:
            create_csv(ticker)

In [316]:
def testing_100ma_20ma(stock='TSLA'):
    
    '''This is for testing when the rolling averages are profitable, mostly just for testing'''
       
    buy_stock=[]
    sell_stock=[]
    sum_total=0
    
    Previous_adj_close=0
    sell_if_can=False
        
    df=pd.read_csv('stock_dfs/Testing/{}.csv'.format(stock),parse_dates=True,index_col=0)
    
    df['300ma']=df['Adj Close'].rolling(window=300,min_periods=0).mean()
    df['100ma']=df['Adj Close'].rolling(window=100,min_periods=0).mean()
    df['40ma']=df['Adj Close'].rolling(window=40,min_periods=0).mean()
    df['20ma']=df['Adj Close'].rolling(window=20,min_periods=0).mean()
    
    for index,row in df.iterrows():
        Current_adj_close=row['Adj Close']
        if row['100ma']<Previous_adj_close and row['100ma']>Current_adj_close and row['20ma']<row['100ma'] and sell_if_can==False:
            buy_stock.append(Current_adj_close)
            sell_if_can = True
        if sell_if_can==True and row['20ma']>Previous_adj_close and row['20ma']<Current_adj_close and row['20ma']>buy_stock[-1]:
            sell_stock.append(Current_adj_close)
            sell_if_can=False
        Previous_adj_close=Current_adj_close

    for i in range(0,len(buy_stock)):
        try:
            #print(sell_stock[i]-buy_stock[i])
            sum_total+=sell_stock[i]-buy_stock[i]
        except IndexError:
            pass
    #print(sum_total)
    return sum_total
    
def testing_100ma_40ma(stock='TSLA'):
    
    '''This is for testing when the rolling averages are profitable, mostly just for testing'''
       
    buy_stock=[]
    sell_stock=[]
    sum_total=0
    
    Previous_adj_close=0
    sell_if_can=False
        
    df=pd.read_csv('stock_dfs/Testing/{}.csv'.format(stock),parse_dates=True,index_col=0)
    
    df['300ma']=df['Adj Close'].rolling(window=300,min_periods=0).mean()
    df['100ma']=df['Adj Close'].rolling(window=100,min_periods=0).mean()
    df['40ma']=df['Adj Close'].rolling(window=40,min_periods=0).mean()
    df['20ma']=df['Adj Close'].rolling(window=20,min_periods=0).mean()
    
    for index,row in df.iterrows():
        Current_adj_close=row['Adj Close']
        if row['100ma']<Previous_adj_close and row['100ma']>Current_adj_close and row['40ma']<row['100ma'] and sell_if_can==False:
            buy_stock.append(Current_adj_close)
            sell_if_can = True
        if sell_if_can==True and row['40ma']>Previous_adj_close and row['40ma']<Current_adj_close and row['40ma']>buy_stock[-1]:
            sell_stock.append(Current_adj_close)
            sell_if_can=False
        Previous_adj_close=Current_adj_close

    for i in range(0,len(buy_stock)):
        try:
            #print(sell_stock[i]-buy_stock[i])
            sum_total+=sell_stock[i]-buy_stock[i]
        except IndexError:
            pass
    #print(sum_total)
    return sum_total
    
def testing_40ma_20ma(stock='TSLA'):
    
    '''This is for testing when the rolling averages are profitable, mostly just for testing'''
       
    buy_stock=[]
    sell_stock=[]
    sum_total=0
    
    Previous_adj_close=0
    sell_if_can=False
        
    df=pd.read_csv('stock_dfs/Testing/{}.csv'.format(stock),parse_dates=True,index_col=0)
    
    df['300ma']=df['Adj Close'].rolling(window=300,min_periods=0).mean()
    df['100ma']=df['Adj Close'].rolling(window=100,min_periods=0).mean()
    df['40ma']=df['Adj Close'].rolling(window=40,min_periods=0).mean()
    df['20ma']=df['Adj Close'].rolling(window=20,min_periods=0).mean()
    
    for index,row in df.iterrows():
        Current_adj_close=row['Adj Close']
        if row['40ma']<Previous_adj_close and row['40ma']>Current_adj_close and row['20ma']<row['40ma'] and sell_if_can==False:
            buy_stock.append(Current_adj_close)
            sell_if_can = True
        if sell_if_can==True and row['20ma']>Previous_adj_close and row['20ma']<Current_adj_close and row['20ma']>buy_stock[-1]:
            sell_stock.append(Current_adj_close)
            sell_if_can=False
        Previous_adj_close=Current_adj_close

    for i in range(0,len(buy_stock)):
        try:
            #print(sell_stock[i]-buy_stock[i])
            sum_total+=sell_stock[i]-buy_stock[i]
        except IndexError:
            pass
    #print(sum_total)
    return sum_total

In [317]:
def add_double_derivative(stock='TSLA'):
    
    '''More testing to see if numerical integration could be useful for finding trends'''
    #This doesn't work properly, fix it later
    
    df=pd.read_csv('stock_dfs/{}.csv'.format(stock),parse_dates=True,index_col=0)
    
    previous_row = df.iloc[0]
    previous_previous_row = df.iloc[0] 
    
    for index,row in df.iterrows():
        #print(previous_previous_row['Adj Close'],previous_row['Adj Close'],row['Adj Close'])
        df['double_derivative']=(float(previous_previous_row['Adj Close'])-2*float(previous_row['Adj Close'])+float(row['Adj Close']))/4
        previous_previous_row=previous_row
        previous_row=row
    #print(df)
    
    '''
    y_spl = UnivariateSpline(np.arange(0,df.shape[0]),df['Adj Close'],s=0,k=4)
    #plt.semilogy(np.arange(0,df.shape[0]),df['Adj Close']) #Check to see if it worked
    y_spl_2d = y_spl.derivative(n=2)
    #plt.plot(np.arange(0,df.shape[0]),y_spl_2d(np.arange(0,df.shape[0])))
    #plt.show()
    df['double_derivative'] = y_spl_2d
    print(df.head())'''

In [318]:
def record_testing_results():
    
    df=pd.DataFrame(columns = ['Stock','100ma - 20ma','100ma - 40ma','40ma - 20ma'])
    
    for file in os.listdir('stock_dfs/Testing/'):
        list_dict=[]
        if file.endswith(".csv") and file!='rolling_average_testing.csv':
            stock = file[:file.find('.')]
            list_dict.append(stock)
            #print(stock)
            list_dict.append(testing_100ma_20ma(stock))
            list_dict.append(testing_100ma_40ma(stock))
            list_dict.append(testing_40ma_20ma(stock))
            df.loc[len(df)]=list_dict
    df.to_csv('stock_dfs/rolling_average_testing.csv')

In [315]:
def main(name):
    
    name=str(name)
    start=dt.datetime(2017,1,1)
    
    #get_data_from_yahoo()
    
    #create_csv(name,start)
    #read_csv(name)
    #plt.show()
    #testing_100ma_20ma(name)
    #testing_100ma_40ma(name)
    #testing_40ma_20ma(name)
    #add_double_derivative(name) #Fix this
    record_testing_results()
    
    
main('MU')

A
0
AAL
3.425065
AAP
0
AAPL
17.248718
ABBV
24.74834
ABC
7.011497
ABT
0.098965
ACN
4.917618
ADBE
24.899994
ADI
17.254684
ADM
4.781825
ADP
8.330962
ADS
10.695267
ADSK
24.199997
AEE
1.440658
AEP
5.29821
AES
0.587037
AET
37.501601
AFL
0
AGN
2.090653
AIG
4.864216
AIV
3.714646
AIZ
6.777931
AJG
3.722043
AKAM
0
ALB
0.706444
ALGN
0
ALK
0
ALL
10.728195
ALLE
8.030311
ALXN
3.570008
AMAT
6.049694
AMD
3.48
AME
1.055771
AMG
32.02884
AMGN
3.039993
AMP
30.609703
AMT
0.650238
AMZN
126.759949
ANDV
0
ANSS
0
ANTM
45.246506
AON
0
AOS
0
APA
0
APC
0
APD
4.488495
APH
0
APTV
12.6428
ARE
2.52166
ARNC
0.857992
ATVI
1.789993
AVB
2.139984
AVGO
47.855593
AVY
0
AWK
2.938912
AXP
0.378464
AYI
8.346558
AZO
0
BA
10.004837
BABA
0
BAC
1.53626
BAX
2.981128
BBT
3.300423
BBY
0
BDX
25.40863
BF-B
0.634839
BHF
0.700001
BHGE
0
BIIB
26.170013
BK
2.379413
BLK
52.532836
BLL
5.48537
BMY
1.824527
BRK-B
4.960006
BSX
3.290001
BTCUSD=X
1682.789062
BWA
0.355332
BXP
5.45948
BZUN
3.899998
CA
0.300567
CAH
5.174689
CAT
2.952393
CBG
5.379999
C